In [1]:
import numpy as np
import pandas as pd
import os
from sklearn import svm
from sklearn import metrics
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats, linalg
import warnings
from sklearn.exceptions import ConvergenceWarning
def conv_r2z(r):
    with np.errstate(invalid='ignore', divide='ignore'):
        return 0.5 * (np.log(1 + r) - np.log(1 - r))
def conv_z2r(z):
    with np.errstate(invalid='ignore', divide='ignore'):
        return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)

# Load Brain data

In [2]:
base_path = os.path.dirname(os.path.dirname(os.getcwd()))
path = base_path + '/data'
nR = 122
thres = 0.01

In [3]:
# load brain
import mat73
brain_sherlock = mat73.loadmat(path + '/brain/' + 'sherlock' + '/a_output/FC/sliding-dynFeat.mat')['dynFeat']
# zscore per feature
brain_sherlock = scipy.stats.zscore(brain_sherlock,2,nan_policy='omit') 
print('  model learn: brain entire feature shape '+str(brain_sherlock.shape))

brain_FNL = mat73.loadmat(path + '/brain/' + 'FNL' + '/a_output/FC/sliding-dynFeat.mat')['dynFeat']
# zscore per feature
brain_FNL = scipy.stats.zscore(brain_FNL,2,nan_policy='omit') 
print('  model learn: brain entire feature shape '+str(brain_FNL.shape))

  model learn: brain entire feature shape (16, 7381, 1894)
  model learn: brain entire feature shape (35, 7381, 1341)


# Feature selection

In [4]:
# load behavior

arousal_FNL = scipy.io.loadmat(path + '/beh/preprocessed/group_average/conv_slidingBeh/FNL_arousal.mat')['sliding_beh']
arousal_FNL = np.squeeze(arousal_FNL)

arousal_sherlock = scipy.io.loadmat(path + '/beh/preprocessed/group_average/conv_slidingBeh/sherlock_arousal.mat')['sliding_beh']
arousal_sherlock = np.squeeze(arousal_sherlock)

print('model learn: FNL arousal shape' + str(arousal_FNL.shape))
print('model learn: sherlock arousal shape' + str(arousal_sherlock.shape))

model learn: FNL arousal shape(1341,)
model learn: sherlock arousal shape(1894,)


In [5]:
filepath = base_path + '/results/sherlock/within_prediction/sliding_arousal.mat'
pos_feat = scipy.io.loadmat(filepath)['pos_feat']
neg_feat = scipy.io.loadmat(filepath)['neg_feat']
pos_feat_s, neg_feat_s = np.average(pos_feat,0), np.average(neg_feat,0)
for i1 in range(nR):
    for i2 in range(nR):
        if pos_feat_s[i1,i2]<1:
            pos_feat_s[i1,i2]=0
        if neg_feat_s[i1,i2]<1:
            neg_feat_s[i1,i2]=0
print(' #pos = '+str(int(np.sum(pos_feat_s)/2)), ', #neg = '+str(int(np.sum(neg_feat_s)/2)))
all_feat_s = pos_feat_s+neg_feat_s
featid_s,featid_s_pos,featid_s_neg  = [],[],[]
ii = -1
for i1 in range(nR-1):
    for i2 in range(i1+1,nR):
        ii=ii+1
        if all_feat_s[i1,i2]==1:
            featid_s.append(ii)
        if pos_feat_s[i1,i2]==1:
            featid_s_pos.append(ii)
        if neg_feat_s[i1,i2]==1:
            featid_s_neg.append(ii)
            
            
filepath = base_path + '/results/FNL/within_prediction/sliding_arousal.mat'
pos_feat = scipy.io.loadmat(filepath)['pos_feat']
neg_feat = scipy.io.loadmat(filepath)['neg_feat']
pos_feat_f, neg_feat_f = np.average(pos_feat,0), np.average(neg_feat,0)
for i1 in range(nR):
    for i2 in range(nR):
        if pos_feat_f[i1,i2]<1:
            pos_feat_f[i1,i2]=0
        if neg_feat_f[i1,i2]<1:
            neg_feat_f[i1,i2]=0
print(' #pos = '+str(int(np.sum(pos_feat_f)/2)), ', #neg = '+str(int(np.sum(neg_feat_f)/2)))
all_feat_f = pos_feat_f+neg_feat_f
featid_f,featid_f_pos,featid_f_neg  = [],[],[]
ii = -1
for i1 in range(nR-1):
    for i2 in range(i1+1,nR):
        ii=ii+1
        if all_feat_f[i1,i2]==1:
            featid_f.append(ii)
        if pos_feat_f[i1,i2]==1:
            featid_f_pos.append(ii)
        if neg_feat_f[i1,i2]==1:
            featid_f_neg.append(ii)

 #pos = 439 , #neg = 154
 #pos = 848 , #neg = 730


In [6]:
overlap_pos, overlap_neg = np.zeros((nR,nR)), np.zeros((nR,nR))

for i in range(nR):
    for j in range(nR):
        if pos_feat_s[i,j] == 1 and pos_feat_f[i,j] == 1:
            overlap_pos[i,j] = 1
        else: 
            overlap_pos[i,j] = 0

        if neg_feat_s[i,j] == 1 and neg_feat_f[i,j] == 1:
            overlap_neg[i,j] = 1
        else: 
            overlap_neg[i,j] = 0
print('Overlap positive feature #: ' + str(int(np.sum(overlap_pos)/2)))
print('Overlap negative feature #: ' + str(int(np.sum(overlap_neg)/2)))

Overlap positive feature #: 169
Overlap negative feature #: 68


In [7]:
featid_pos = [i for i in featid_f_pos if i in featid_s_pos]
featid_neg = [i for i in featid_f_neg if i in featid_s_neg]
featid = np.append(featid_pos,featid_neg)
print('Selected ' + str(len(featid)) +' features')

Selected 237 features


In [8]:
# select FCs in the brain data
feat_sherlock = brain_sherlock[:,featid,:]
feat_FNL = brain_FNL[:,featid,:]

print('  model sherlock: brain selected feature shape '+str(feat_sherlock.shape))
print('  model FNL : brain selected feature shape '+str(feat_FNL.shape))

  model sherlock: brain selected feature shape (16, 237, 1894)
  model FNL : brain selected feature shape (35, 237, 1341)


# Process the brain and behavior for SVM

In [9]:
nsubj_s = 16
nsubj_f = 35

s_feat = np.transpose(feat_sherlock,(1,0,2))
s_feat = np.reshape(s_feat,(s_feat.shape[0],s_feat.shape[1]*s_feat.shape[2]))

f_feat = np.transpose(feat_FNL,(1,0,2))
f_feat = np.reshape(f_feat,(f_feat.shape[0],f_feat.shape[1]*f_feat.shape[2]))

print('Reshape brain sherlock ' + str(s_feat.shape))
print('Reshape brain FNL ' + str(f_feat.shape))

s_arousal = []
for sub in range(nsubj_s):
    s_arousal.append(arousal_sherlock)
s_arousal = np.asarray(s_arousal)
s_arousal = np.reshape(s_arousal, (s_arousal.shape[0])*s_arousal.shape[1])

f_arousal = []
for sub in range(nsubj_f):
    f_arousal.append(arousal_FNL)
f_arousal = np.asarray(f_arousal)
f_arousal = np.reshape(f_arousal, (f_arousal.shape[0])*f_arousal.shape[1])

print('Reshape beh sherlock ' + str(s_arousal.shape))
print('Reshape beh FNL ' + str(f_arousal.shape))

# concatenate
train_feat = np.concatenate([f_feat,s_feat],axis=1)
train_beh = np.concatenate([f_arousal,s_arousal])

print('Combined brain ' + str(train_feat.shape))
print('Combined beh ' + str(train_beh.shape))

Reshape brain sherlock (237, 30304)
Reshape brain FNL (237, 46935)
Reshape beh sherlock (30304,)
Reshape beh FNL (46935,)
Combined brain (237, 77239)
Combined beh (77239,)


In [10]:
# if several TRs are removed
rmtr_train = []
for tm in range(train_feat.shape[1]):
    if np.all(np.isnan(train_feat[:,tm])):
        rmtr_train.append(tm)
rmtr_train = np.asarray(rmtr_train)
if len(rmtr_train)>0:
    train_feat = np.delete(train_feat,rmtr_train,1)
    train_beh = np.delete(train_beh,rmtr_train,0)
print('Number of TR removed: '+str(len(rmtr_train)))

Number of TR removed: 0


# Load Merlin

In [11]:
#load brain

brain_merlin = mat73.loadmat(path + '/brain/' + 'Merlin' + '/a_output/FC/sliding-dynFeat.mat')['dynFeat']
# zscore per feature
brain_merlin = scipy.stats.zscore(brain_merlin,2,nan_policy='omit') 
print('  model learn: brain entire feature shape '+str(brain_merlin.shape))

  model learn: brain entire feature shape (18, 7381, 996)


In [12]:
# load beh
arousal_merlin = scipy.io.loadmat(path + '/beh/preprocessed/group_average/conv_slidingBeh/Merlin_arousal.mat')['sliding_beh']
arousal_merlin = np.squeeze(arousal_merlin)
print('model learn: merlin arousal shape' + str(arousal_merlin.shape))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/cablab/Desktop/AffectPrediction/data/beh/preprocessed/group_average/conv_slidingBeh/Merlin_arousal.mat'

In [ ]:
# delete the first 16 TRs to match behavior with brain according to instructions on openneuro
# https://openneuro.org/datasets/ds001110/versions/00003

dynFC_test = brain_merlin[:,featid,16:]

nanidx = []
for ft in range(train_feat.shape[0]):
    if np.any(np.isnan(train_feat[ft,:])):
        nanidx.append(ft)
for subj in range(dynFC_test.shape[0]):
    for ft in range(dynFC_test.shape[1]):
        if np.any(np.isnan(dynFC_test[subj,ft,:])):
            nanidx.append(ft)

nanidx = np.unique(nanidx)
print('NaN = '+str(nanidx))

if len(nanidx)>0:
    train_feat = np.delete(train_feat,nanidx,0)
    dynFC_test = np.delete(dynFC_test,nanidx,1)

print('Number of nans in selected FC: '+str(len(nanidx)))
print('Train Feat Shape: '+str(train_feat.shape))
print('DynFC_test Shape: '+str(dynFC_test.shape))

# Support Vector Regression with non-linear Kernel

In [ ]:
print('SVR prediction with model learned from FNL + Sherlock')
print('  train feature   : '+str(train_feat.T.shape))
print('  train arousal: '+str(train_beh.shape))

clf = []
clf = svm.SVR(kernel='rbf',max_iter=1000, gamma='auto')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    clf.fit(train_feat.T, train_beh)
print('Training done!')

In [ ]:
test_nsubj = 18
output_acc, output_eval = [], []
for test_sub in range(test_nsubj):
    test_feat = dynFC_test[test_sub, :, :]
    test_behavior = arousal_merlin
    
    rmtr_test = []
    for tm in range(test_feat.shape[1]):
        if np.all(np.isnan(test_feat[:, tm])):
            rmtr_test.append(tm)
    rmtr_test = np.asarray(rmtr_test)
    if len(rmtr_test) > 0:
        test_feat = np.delete(test_feat, rmtr_test, 1)
        test_behavior = np.delete(test_behavior, rmtr_test, 0)
    
    predicted = clf.predict(test_feat.T)
    output_acc.append(predicted)
    
    # evaluate
    pearsonr = scipy.stats.pearsonr(test_behavior, predicted)
    mse = metrics.mean_squared_error(test_behavior, predicted)
    rsq = metrics.r2_score(test_behavior, predicted)
    output_eval.append([pearsonr[0], mse, rsq])
    
    print(' subj ' + str(test_sub + 1) + ' / ' + str(test_nsubj) + ': pearson r=' + str(
            np.round(pearsonr[0], 3)), ', mse=' + str(np.round(mse, 3)) + ', rsq=' + str(np.round(rsq, 3)))
    print('              (train) ft ' + str(train_feat.shape[1]) + ', beh ' + str(train_beh.shape[0]) +
              ', (test) ft ' + str(test_feat.shape[1]) + ', beh ' + str(test_behavior.shape[0]))
    
output_acc, output_eval = np.asarray(output_acc), np.asarray(output_eval)

In [ ]:
fig, axs = plt.subplots(1,3)
axs[0].hist(output_eval[:,0])
axs[0].set_title('Pearson correlation')
axs[1].hist(output_eval[:,1])
axs[1].set_title('MSE')
axs[2].hist(output_eval[:,2])
axs[2].set_title('r-squared')

print(' pearson r = '+str(np.round(conv_z2r(np.mean(conv_r2z(output_eval[:,0]))),3)))
print(' MSE       = '+str(np.round(np.mean(output_eval[:,1]),3)))
print(' r-squared = '+str(np.round(np.mean(output_eval[:,2]),3)))

In [ ]:
savepath = base_path + '/results/CPM/across/'
result = {'acc':output_acc, 'eval':output_eval}
if os.path.exists(savepath)==0:
    os.makedirs(savepath)
scipy.io.savemat(savepath+'overlap-Merlin_arousal.mat',result)